<a href="https://colab.research.google.com/github/ilitteri/7506-Datos/blob/main/tp1_parte1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [7]:
def path_for(dataset: str) -> str:
    return f'/content/drive/MyDrive/{dataset}.csv'

business = pd.read_csv(path_for("Dataset/business"))
review = pd.read_csv(path_for("Dataset/review"))

#  P13

Nos vamos a quedar dos días en New Orleans por una meetup de data science. Queremos visitar la ciudad pero no tenemos mucho tiempo así que visitaremos sus mejores lugares. Vamos a calcular el score de review promedio para cada lugar, pero para tener en cuenta la varianza vamos a restarle a cada promedio su desviación estándar y solo usar lugares con más de 10 reviews.

- Nos han dicho que la ciudad tiene un barrio francés con muy buena gastronomía. ¿Cúal es el mejor lugar para comer con la categoría “French”?
- Después de comer queremos ir a un bar a tomar tragos, ¿cuál es el mejor de la categoría “Bars”?
- ¿Cuál es el mejor museo (categoría “Museums”)? ¿De qué trata?


In [28]:
cantidad_de_reviews_por_lugar = review[['business_id', 'stars']].groupby('business_id').count()
lugares_con_mas_de_10_reviews = cantidad_de_reviews_por_lugar[cantidad_de_reviews_por_lugar['stars'] > 10].reset_index()['business_id']
estrellas_de_reviews_agrupadas_por_lugares = review.merge(lugares_con_mas_de_10_reviews)[['business_id', 'stars']].groupby('business_id')
score_de_estrellas_promedio = estrellas_de_reviews_agrupadas_por_lugares.mean() - estrellas_de_reviews_agrupadas_por_lugares.std()
score_de_estrellas_promedio.columns = ['score']
score_de_estrellas_promedio = score_de_estrellas_promedio.reset_index()
lugares_con_score = business.merge(score_de_estrellas_promedio)
lugares_con_score.categories = lugares_con_score.categories.map(lambda c: str(c) if not type(c) == str else c)

def es_de_categoria(categoria: str, categorias: str):
    return categoria in categorias.split(', ')

def encontrar_mejor_lugar_para_categoria(categoria: str, df):
    return df[df['categories'].notnull() & df.categories.map(lambda categorias: es_de_categoria(categoria, categorias))].sort_values('score', ascending=False).head(1)


mejor_museo = encontrar_mejor_lugar_para_categoria(categoria='Museums', df=lugares_con_score).reset_index()
otras_categorias_de_mejor_museo = business[business['business_id'] == mejor_museo['business_id'][0]]['categories'].str.split(', ').tolist()[0]
otras_categorias_de_mejor_museo.remove('Museums')

print(f"El mejor lugar para comer con la categoría “French” es: {encontrar_mejor_lugar_para_categoria(categoria='French', df=lugares_con_score).reset_index()['name'][0]}")
print(f"El mejor lugar para ir a tomar algo con la categoría “Bars” es: {encontrar_mejor_lugar_para_categoria(categoria='Bars', df=lugares_con_score).reset_index()['name'][0]}")
print(f"El mejor museo es: {mejor_museo['name'][0]} y también pertenece a las categorías de:", *otras_categorias_de_mejor_museo)

El mejor lugar para comer con la categoría “French” es: Bistro La Bête
El mejor lugar para ir a tomar algo con la categoría “Bars” es: Jerry's Cigars
El mejor museo es: Lower Ninth Ward Living Museum y también pertenece a las categorías de: Arts & Entertainment
